In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## The Mystic Art of Prompting - Advanced Techniques

In [ ]:
# First let's install some packages, import stuff and define our generation model

!pip install langchain
!pip install google-search-results
!pip install numexpr

from vertexai.language_models import TextGenerationModel

generation_model = TextGenerationModel.from_pretrained("text-bison@001") # if you are feeling experimental - try text-bison@002! 

### Chain of Thought

Chain of Thought (CoT) [6] prompting is a technique which was first introduced in January 2022 for improving the reasoning capabilities of LLMs by letting them write intermediate reasoning steps. This helps the LLM understand the problem more deeply, and generate more accurate answers. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding.


In [ ]:
#Lets first look at the example without CoT

prompt = """When I was 3 years old, my partner was 3 times my age. Now, I am 20 years old. How old is my partner?

"""

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

In [ ]:
#Now, lets add CoT

prompt = """ When I was 3 years old, my partner was 3 times my age. Now, I am 20 years old. How old is my partner? Let's think step by step.
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

Chain of Thought prompting can be very powerful when combined with a single-shot

In [ ]:
#Now, lets add CoT

prompt = """ Q: When my brother was 4 years old, I was double his age. Now I am 40 years old. How old is my brother? Let's think step by step.
A: When my brother was 4 years, I was 2 * 2 = 8 years old. That's an age difference of 4 years and I am older. Now I am 40 years old, so my brother is 40 - 4  = 36 years old. The answer is 36.
Q: When I was 3 years old, my partner was 3 times my age. Now, I am 20 years old. How old is my partner? Let's think step by step.
A:
"""
print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

### Tree of Thoughts

Now that we are familiar with Chain of Thoughts, let's review Tree of Thoughts (ToT) , which was first introduced in May 2023. It generalizes the concept of CoT prompting because it allows LLMs to explore multiple different reasoning paths simultaneously, rather than just following a single linear chain of thought.

![Tree of Thoughts](https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2FTOT.3b13bc5e.png&amp;w=3840&amp;q=75)



In [ ]:
prompt = """The question is: "When I was 3 years old, my partner was 3 times my age. Now, I am 20 years old. How old is my partner?"

Now imagine 3 different experts are answering this question.
All experts will write down the steps of their thinking, by calculating the age difference back then to calculate the current age of the partner. If any expert realizes they're wrong at any point then they leave.
The correct answer is the answer that has been answered the most.

"""

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

### Reason and Act

Reason and Act (ReAct) [8] prompting was first introduced in October 2022 and is a paradigm for enabling LLMs to solve complex tasks using natural language reasoning. It is designed for tasks where the LLM is allowed to perform certain actions, such as interacting with external APIs to retrieve information.
ReAct mimics how humans operate in the real world, as we reason verbally and can take actions to gain information. ReAct performs well against other prompt engineering approaches in a variety of domains.

ReAct prompting works by combining reasoning and acting into a thought-action loop. The LLM first reasons about the problem and generates a plan of action. It then performs the actions in the plan and observes the results. The LLM then uses the observations to update its reasoning and generate a new plan of action. This process continues until the LLM reaches a solution to the problem.


In [ ]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import VertexAI

#Create a SERPAPI_API_KEY on https://serpapi.com/ for free to use (amongst others) the Google Search API
SER_API_KEY = "[YOUR_SERPAPI_API_KEY]"
import os
os.environ['SERPAPI_API_KEY']=str(SER_API_KEY)


from langchain.utilities import SerpAPIWrapper

search = SerpAPIWrapper()




prompt = "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"

llm = VertexAI(temperature=0.1)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

agent = initialize_agent(tools, llm,  agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run(prompt)


### Automatic Prompt Engineering

At this point you might realize that writing a prompt can be complex and. Wouldn't it be possible to automate this (write a prompt to write prompts) and take out natural language ambiguity? Well, there's a method: Automatic Prompt Engineering (APE). . This method - described in a 2023 research paper  - not only alleviates the need for human input but also enhances the model’s performance in various tasks.

The idea is at follows:

You pick a task that needs to automate the prompt for crafting and optimizing the best prompt for you.

In our example, we are building a shopping assistant for a retail company. We want to figure out all the various ways customers could phrase their order for buying a new coat.

Write the prompt which will generate the instruction candidates. You can do this through an LLM, but there are also various hubs and collections of prompts that you can find online. In this example, I am using Text-Bison-32k to generate 10 instructions.


In [ ]:
prompt = """We have a retail webshop, and to train a chatbot we need various ways to order: "Coat, black, puffer jacket size M". Generate 10 variants, with the same semantics but keep the same meaning.
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=1024).text)

Now it's your turn! Create a prompt to create a instruction candidate for the perfect prompt for your specific problem!

In [ ]:
prompt = """YOUR PROMPT GOES HERE!
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=1024).text)

better_prompt=(generation_model.predict(prompt=prompt, max_output_tokens=1024).text)

print(generation_model.predict(prompt=better_prompt, max_output_tokens=1024).text)

### Take a step-back prompting

As we have seen with most prompting techniques published, Large Language Models (LLMs) need guidance when intricate, multi-step reasoning is demanded from a query, and decomposition is a key component when solving complex request.

A process of supervision with step-by-step verification is a promising remedy to improve the correctness of intermediate reasoning step

The most well known prompting technique when it comes to decomposition is chain-of-thought reasoning. In this study Step-Back Prompting is compared to COT prompting.

The text below shows a complete example of STP with the original question, the stepback question, principles, and the prompt for the final answer to be generated by the LLM.

In [ ]:
#This shows example of the stepback question that is generated

prompt = """ You are an expert at world knowledge.
Your task is to step back and paraphrase a question to a more generic
step-back question, which is easier to answer.

Here are a few examples:
Original Question: Which position did Knox Cunningham hold from May 1955 to Apr 1956?
Stepback Question: Which positions have Knox Cunning- ham held in his career?

Original Question: Who was the spouse of Anna Karina from 1968 to 1974?
Stepback Question: Who were the spouses of Anna Karina?

Original Question: Which team did Thierry Audel play for from 2007 to 2008?
Stepback Question: Which teams did Thierry Audel play for in his career?

Question: Was donald trump president when Palm was announced?
"""

print(generation_model.predict(prompt=prompt, max_output_tokens=1024, temperature=0.1).text)

In [ ]:
from langchain.chat_models import ChatVertexAI
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda

In [ ]:
question = "did Google Bard exist while Trump was president? Answer with yes or no and provide your explanation."


In [ ]:
def retriever(query):
    return search.run(query)

response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

{normal_context}

Original Question: {question}
Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

######################################

chain = {
    # Retrieve context using the normal question
    "normal_context": RunnableLambda(lambda x: x['question']) | retriever,
    # Pass on the question
    "question": lambda x: x["question"]
} | response_prompt | ChatVertexAI(temperature=0) | StrOutputParser()

######################################

chain.invoke({"question": question})

## There can be only one!

Now it is time to put all that theory into practice! First have a look at the additional best pratices below:

**Emphasis through prompting:**
Emphasis through Capitalization: Use capitalization to stress important points or instructions. This can help guide the model's responses and maintain a certain focus. For instance, "Remember to KEEP DETAILS CONFIDENTIAL".

**Extreme Emphasis:**
You can use exaggerations or hyperbolic language to amplify the importance of your prompt or instructions. For instance, instead of saying "Make sure it's clear," you might say, "Your explanation should be as clear as a summer's day, absolutely impossible to misinterpret. Every single word must ooze clarity!"

**Emphasis through Repetition:**
Reiteration of key phrases or ideas can effectively guide the AI model's responses. For example, "remember to be creative... keep in mind to be creative... don't forget to be creative..."

**Prompt modularization:**
Modularize instruction: Break down complex tasks into a sequence of simpler prompts, facilitating more interactive and manageable conversations. Ex: Instead of having many instructions in 1 prompt, modularize the prompts and have 1 prompt per instruction. Then based on the user’s  input, you can choose which prompt to process.

**Prompt chaining:**
This is a technique to complete complex tasks by chaining together multiple prompts. The output of one prompt is used as the input for the next prompt, and so on. For example, first convert the file format to csv, then do operation x, then do operation y on the new dataset and so on.

**Aggregation technique:**
This is a technique to perform certain tasks on specific portions of the data and then aggregate them all to produce the final output. Ex: Do operation x on the first part of the data, do operation y on the rest of the data and aggregate the results.

#### Trial and error:
**Iterative prompting:**
Technique where prompts are continuously refined and adjusted to enhance clarity, effectiveness, and quality when working with large language models; by progressively adapting prompts based on feedback, it ensures that responses meet desired criteria; this is especially useful in application development and content generation, optimizing interaction between human input and machine learning models for better outcomes.

**Starting over:**
Sometimes, when a prompt is not working well, and some iterations of it are not doing it, start from scratch with a different prompt, see which sections from the previous prompt help, which don’t - filter out the sections which “work” (make the model behave the way you want) and add to that.

**Modular testing:**
When a prompt is not working, split the task into many sub tasks and run the portion of the prompt that achieves a given sub-task. This will help you identify where it is not working and fix the prompt accordingly.

#### Reinforcement techniques:
**Redundancy Technique:**
Similar to the "Capitalize vs Repeat" technique, use synonyms or different phraseologies for the same instruction to reinforce the direction of the prompt. Ex: Instead of saying ‘make sure to..’ you say the same thing many times by replacing ‘make sure’ with ‘ensure’, ‘guarantee’, ‘verify’..  

**Negative Instructions:**
Explicitly mention what you don't want in the response. This is useful when the model tends to produce certain unwanted outputs for specific prompts. Ex: ‘Don’t output any sources.’

**Self-reference:**
Use the AI's self-awareness to your advantage by instructing it to evaluate or check its own responses before producing them. For example, "Make sure your response is concise and does not include unnecessary details".

#### Prompt formatting techniques:
**Structure:**
Having a structured prompt helps the model better understand the task. Start by defining its role. Ex: “Your role is to read the input data and structure it in the following format” Then tell it what the input data is, then tell it what the desired format is, then tell it how to do it.

**Use constraints:**
Constraints can be used to limit the scope of the model's output. For example, you could constrain the model to generate a line that is no longer than 10 words long or not to output any other data than the input data.

**Use delimiters:**
Delimiters in prompt engineering provide clear distinctions within your text, effectively separating instructions from content. This practice helps prevent misunderstandings, allows you greater control over AI responses, and ensures consistent results, even when handling complex tasks.

#### Use LLMs (ex: Bard) to build prompts:
**Refining Prompts:**
By providing an initial prompt and the output received, you can ask the LLM to refine the prompt to achieve a more desired output. The LLM can provide insight into what might be improved, suggest changes

**Generating New Prompts:**
If you provide the LLM with a set of requirements, desired output format, and examples of "requirement, prompt" tuples, it can generate a new prompt based on the given requirements. This could be useful when creating prompts for new scenarios or tasks.

**Feedback:**
Once you have a prompt, you can ask the LLM for feedback. This can help you to identify areas where the prompt could be improved. The LLm can help you point out some gaps in your prompt engineering skills based on your prompt and what you want to achieve.

#### Miscellaneous:
**Prioritization:**
Clearly communicate to the AI model what content or information is most critical. This approach helps in guiding the AI's focus and ensures the most important topics or issues are addressed accurately.

**Use examples:**
Providing examples of the desired output can also be helpful. This can help the model to understand what you are looking for and to generate more accurate results. Sometimes you can describe what you want so well that you won’t need an example, but it only helps the model to give it more information as long as you’re operating below the context window.

**Preferred Output Format:**
Specify your desired output format. If a CSV format is more useful than plain text, instruct the AI model to produce the output in that format from the outset. This strategy can save time and effort on manual conversions later.
We have seen that the model processes csv and json format better than it does with txt files.
Prompting it to first convert the txt file to csv or json yields a better result.
Writing your prompt in json format or requesting the output to be in json or csv format yield more accurate results.

**Use a prompt library:**
There are a number of online libraries that contain pre-made prompts that can be used for a variety of tasks. We have a prompt library in Vertex AI → Generative AI studio → Language → prompt examples. This can be a helpful resource if you’d like a starting point or have a very common task.

**Nice words:**
Using words like ‘please’, ‘great job!’, ‘you are the best!’ … increases the model’s compliance rate. This increases the ‘confidence level’ of the model and it positively reinforces the model’s behavior which tends to increase accuracy for simple tasks.

**Lastly and most important: Be Creative:**
This isn’t even the tip of the iceberg - the more creative you are, the better you can make the model behave the way you want. For example, instead of saying "provide deep insights," you can write: “Ensure that you provide deep insights... Provide very very deep insights. I told you to give deep insights!!!!! Do not write anything that is not deeply insightful! Make sure to check this requirement is 100% met every time you respond. It’s better that you don’t return anything than returning something that’s not very very insightful!!!”. If you have a long prompt, you may also consider adding the same statement in different places - this helps the model prioritize that task (aka sandwich technique).



Advanced Assignment

## Additional Exercises


- Try out https://gandalf.lakera.ai/  Cannot find the password? Tips: https://medium.com/the-abcs-of-ai/gandalfs-challenge-mastering-prompt-engineering-for-ai-success-fd777be2aa0b
- Try out the examples